In [ ]:
!pip install polars numpy implicit tqdm

In [34]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100


In [36]:
train = pl.read_parquet(...)
test = pl.read_parquet(...)
vacancies = pl.read_parquet(...)

In [37]:
# train = train.to_pandas()
# test = test.to_pandas()
# vacancies = vacancies.to_pandas()

In [20]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"


In [21]:
test

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]"


In [22]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""<strong>Обязан…","[""Пользователь ПК"", ""Работа в команде"", … ""PR-консультирование""]",16500,null,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""<p><strong>Воз…","[""Активные продажи"", ""Холодные продажи"", … ""Организация мероприятий""]",50000,null,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""<strong>Обязан…",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""<p><strong>Для…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""<p>Компания ТД…","[""Продуктивность"", ""Клиентоориентированность"", … ""Развитие продаж""]",60000,null,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""<p><strong>X</…","[""Контроль качества"", ""Производственный контроль"", … ""Лабораторная диагностика""]",88000,null,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""<p>Если ты ище…","[""Активные продажи"", ""Поиск и привлечение клиентов"", … ""Проведение переговоров""]",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""<strong>Обязан…",null,null,null,null,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""<em><strong>Ст…",null,100000,null,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [23]:
predictions = test.select(pl.col('vacancy_id').list.unique(maintain_order=True).list.tail(2))['vacancy_id'].to_list()
test = test.with_columns(pl.lit(predictions).alias('predictions'))

test.select(['user_id', 'session_id', 'predictions']).write_parquet('submission.pq')

/tmp/ipykernel_583187/2589823886.py:2: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  test = test.with_columns(pl.lit(predictions).alias('predictions'))


In [24]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2597196"", ""v_1223061""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_470400"", ""v_1530783""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1592343"", ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1375331"", ""v_1922852""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_2152997"", ""v_1217630""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_698513"", ""v_212060""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_1828011"", ""v_2297339""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_206700"", ""v_528671""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_36093"", ""v_1500697""]"


# Simple ALS

In [25]:
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

pairs = train.select(['user_id', 'vacancy_id', 'action_type']).explode(('vacancy_id', 'action_type'))
users = pairs['user_id'].map_dict(user2idx).to_numpy()
vacancies = pairs['vacancy_id'].map_dict(vac2idx).to_numpy()
preferences = pairs['action_type'].map_dict(action_weights).to_numpy()

/tmp/ipykernel_583187/3619716076.py:15: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  users = pairs['user_id'].map_dict(user2idx).to_numpy()
/tmp/ipykernel_583187/3619716076.py:16: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  vacancies = pairs['vacancy_id'].map_dict(vac2idx).to_numpy()
/tmp/ipykernel_583187/3619716076.py:17: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  preferences = pairs['action_type'].map_dict(action_weights).to_numpy()


In [33]:
pairs

user_id,vacancy_id,action_type
str,str,i64
"""u_332060""","""v_2571684""",2
"""u_332060""","""v_488179""",2
"""u_332060""","""v_2389179""",2
"""u_332060""","""v_1393783""",2
"""u_332060""","""v_2608935""",2
"""u_332060""","""v_1119127""",2
"""u_332060""","""v_323088""",1
"""u_332060""","""v_794390""",1
"""u_332060""","""v_1020162""",2


In [26]:
uv_mat = csr_matrix((preferences, (users, vacancies)))

als_model = implicit.als.AlternatingLeastSquares(
    factors=150,
    random_state=RANDOM_STATE,
    iterations=20,
    calculate_training_loss=True,
    regularization=0.001
)
als_model.fit(uv_mat)

100%|██████████| 20/20 [00:24<00:00,  1.21s/it, loss=2.14e-5]


In [27]:
als_model.user_factors.shape, als_model.item_factors.shape

((882409, 150), (1458353, 150))

In [28]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [30]:
predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in user2idx:
        predictions.append(vacs)
        continue
    cuser = user2idx[user]

    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

100%|██████████| 83189/83189 [02:21<00:00, 588.66it/s]


In [31]:
test = test.with_columns(pl.lit(predictions).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

/tmp/ipykernel_583187/2455226998.py:1: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  test = test.with_columns(pl.lit(predictions).alias('predictions'))


In [32]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2650515"", ""v_367301"", … ""v_2337447""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1347362"", ""v_1798396"", … ""v_726002""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1572055"", ""v_953153"", … ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_2650515"", ""v_367301"", … ""v_676801""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_1431178"", ""v_2395102"", … ""v_1352210""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_1465910"", ""v_1586766"", … ""v_668735""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_701974"", ""v_1081658"", … ""v_27909""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_964765"", ""v_460169"", … ""v_9584""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1431178"", ""v_2395102"", … ""v_2192945""]"
